In [1]:
import pandas as pd

import gc
####

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification
## felipe


from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
!python -m spacy download es_core_news_sm

%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

# etl

In [2]:
import pandas as pd
import sqlite3

In [3]:


query1 = "SELECT NUMERO_PQR,DESCRIPCION_PQR FROM pqr_priorizar where MEDIO_RECEPCION !=  'VIRTUAL' and unico=1 and SINTOMA_PQR ='CONCEPTOS FACTURADOS' and FECHA_CREACION > '2023-06-25' and TECNOLOGIA ='FO'"

query2 = "(SELECT * FROM altas where ST_TIPO_SERVICIO='FTTH' AND FASE='ORDERF' AND (tipoTramite= 'VTA' OR tipoTramite=  'Sales')) AS myquery"

In [6]:

# Connect to the SQLite database file
conn = sqlite3.connect('/content/on2/pqr_priorizar.db')

# Read the SQL query into a pandas DataFrame

pqrs = pd.read_sql_query(query1, conn)

# Close the database connection
conn.close()

# Print the DataFrame
pqrs

#MDM-PQR-39191683

,NUMERO_PQR,DESCRIPCION_PQR
0,MDM-PQR-39565361,CLIENTE SE COMUNICA POR INFORMACIÓN SOBRE SU F...
1,MDM-PQR-39565476,Cliente ARMANDO FARFAN GARZON Cédula de Ciudad...
2,MDM-PQR-39565505,Linea: 6016833978 Cuenta: 712537 Nombre: ARM...
3,MDM-PQR-39565628,Línea Fija: 6016217252 Nombre: RAFAEL GAMBOA C...
4,MDM-PQR-39565657,NURY EDELMIRA MORENO MORENO //51563810 //60136...
...,...,...
3058,MDM-PQR-39736985,NOMBRE: SAMIR MEJIA CC:79316942 CORREO:zamirt...
3059,MDM-PQR-39737146,envio de factura correcta ya que en extemprane...
3060,MDM-PQR-39737246,Cliente EDILIA TOSCANO CORDERO Cédula de Ciuda...
3061,MDM-PQR-39737284,NombreMARIA NELLY RODRIGUEZ ANDRADE Celular 3...


# TRASNFORMAR LAS DATOS DE TEXTO

In [7]:
sys.path.append('../librerias')

In [8]:
from clean_text import clean_text

entro2


In [9]:
pqrs = pqrs.fillna("vacio")



In [10]:
pqrs.isnull().sum()

NUMERO_PQR         0
DESCRIPCION_PQR    0
dtype: int64

In [20]:
#declare answers and questions
descripcion=pqrs.DESCRIPCION_PQR





#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]
clean_descripcion = [clean_text(des) for des in descripcion]


len(clean_descripcion)

3063

In [21]:
clean_descripcion[190]

'le brindaron beneficio de tarifa  pero le expiden factura con distinto valor'

spelling

In [22]:
from tqdm import tqdm
from spelling import correct_sentence


In [23]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

clean_descripcion=corrected_sentences

Processing: 100%|██████████| 3063/3063 [10:12<00:00,  5.00it/s]


lemma token stemm

In [25]:
from lemastem import data_preprocessing

In [26]:

clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[190]

['brindar', 'beneficio', 'tarifa', 'expidir', 'factura', 'distinto']

In [27]:
clean_descripcion[0]

['factura', 'concepto', 'aclarar']

In [28]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


In [29]:
from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)

In [30]:
merged_df

,descripcion
0,factura concepto aclarar
1,factura caer llamado terminar verificacion
2,contrato hacer tiempo indicar primero esro gra...
3,desear validar factura migrar fibra m factura ...
4,poder generar ajuste decir parecer pagar llama...
...,...
3058,ajuste factura falla hacer
3059,parecer
3060,kdbvpp desear validar inactivo
3061,envio facturacion ajustado



clean_descripcion = [text for text in clean_descripcion if "svas" not in text and "directv" not in text  and "winsport" not in text and "hbo" not in text  ]

len(clean_descripcion)

In [31]:
from tfidf import TFIDF
import joblib

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from joblib import dump


X = merged_df['descripcion']
X_test=merged_df['descripcion']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_binario.joblib')

#pca = PCA(n_components=585)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 1571 features
train with old features:  (3063,)
train with new features: (3063, 784)


# cargar modelo

In [33]:
from joblib import dump

import joblib


model = joblib.load('/content/PRIORIZAR/modelo/modelo_binario.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.7487452070745508, bagging_freq=1,
               feature_fraction=0.7904508631838858,
               lambda_l1=0.010538661525506412, lambda_l2=4.2105515010394745,
               learning_rate=0.04002333100026529, min_child_samples=74,
               n_estimators=200)


In [34]:
y_pred = model.predict(X_train_new)

pqrs['prediccion_algoritmo'] = y_pred

pqrs['descripcion_clean']=merged_df['descripcion']



# Lista de palabras a filtrar
words_to_remove = ['svas', 'directv', 'paramont', 'hbo']

# Filtrar las filas del DataFrame
c_facturados_correcion = pqrs[~pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]

c_facturados_SVAS = pqrs[pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]



c_facturados_correcion


,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-39565361,CLIENTE SE COMUNICA POR INFORMACIÓN SOBRE SU F...,1.0,factura concepto aclarar
1,MDM-PQR-39565476,Cliente ARMANDO FARFAN GARZON Cédula de Ciudad...,0.0,factura caer llamado terminar verificacion
2,MDM-PQR-39565505,Linea: 6016833978 Cuenta: 712537 Nombre: ARM...,1.0,contrato hacer tiempo indicar primero esro gra...
3,MDM-PQR-39565628,Línea Fija: 6016217252 Nombre: RAFAEL GAMBOA C...,1.0,desear validar factura migrar fibra m factura ...
4,MDM-PQR-39565657,NURY EDELMIRA MORENO MORENO //51563810 //60136...,1.0,poder generar ajuste decir parecer pagar llama...
...,...,...,...,...
3058,MDM-PQR-39736985,NOMBRE: SAMIR MEJIA CC:79316942 CORREO:zamirt...,0.0,ajuste factura falla hacer
3059,MDM-PQR-39737146,envio de factura correcta ya que en extemprane...,1.0,parecer
3060,MDM-PQR-39737246,Cliente EDILIA TOSCANO CORDERO Cédula de Ciuda...,1.0,kdbvpp desear validar inactivo
3061,MDM-PQR-39737284,NombreMARIA NELLY RODRIGUEZ ANDRADE Celular 3...,1.0,envio facturacion ajustado


In [39]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename1 = f"{root_path}c_facturados_correcion_{current_date}.csv"
filename2 = f"{root_path}c_facturados_SVAS_{current_date}.csv"


c_facturados_correcion.to_csv(filename1, index=False)

c_facturados_SVAS.to_csv(filename2, index=False)




#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE


# modelo retipificacion

filter data with 0 label

In [26]:
retipificacion = c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0]
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-39565361,CLIENTE SE COMUNICA POR INFORMACIÓN SOBRE SU F...,0,unicar factura concepto aclarar
4,MDM-PQR-39565657,NURY EDELMIRA MORENO MORENO //51563810 //60136...,0,poder generar ajuste decir parecer pagar llama...
6,MDM-PQR-39566135,Línea: 6013644591 Nombre: ANA BETRIZ OSORIO CU...,0,solicitar saber llegar factura tanto si decir ...
7,MDM-PQR-39566173,Nombre: MARTIN FLOREZ MORALES CC: 80368377 C...,0,facturacion
9,MDM-PQR-39566352,Nombre: JOSE DELFIN TOVAR CC: 19375096 Celular...,0,
...,...,...,...,...
3055,MDM-PQR-39736638,CLIENTE SOLICITA ACLARAR LOS CONCEPTOS FACTURA...,0,
3057,MDM-PQR-39736819,Línea: 6014104197 Nombre del Cliente: GUILLE...,0,parecer querer saber si aparecer
3059,MDM-PQR-39737146,envio de factura correcta ya que en extemprane...,0,parecer
3061,MDM-PQR-39737284,NombreMARIA NELLY RODRIGUEZ ANDRADE Celular 3...,0,envio facturacion ajustado


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



X = retipificacion['descripcion_clean']
X_test=retipificacion['descripcion_clean']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_retipificacion.joblib')

#pca = PCA(n_components=361)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 836 features
train with old features:  (1180,)
train with new features: (1180, 299)


In [28]:
model = joblib.load('/content/PRIORIZAR/modelo/retipificacion.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.9030872285178855, bagging_freq=3,
               feature_fraction=0.8256242530913764,
               lambda_l1=0.049148437664035254, lambda_l2=1.8266591621558763,
               learning_rate=0.09934154462012913, min_child_samples=23,
               n_estimators=41, num_leaves=33)


In [29]:
y_pred = model.predict(X_train_new)

retipificacion["retipificacion_algoritmo"]=y_pred

<ipython-input-29-9505f33dc576>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion["retipificacion_algoritmo"]=y_pred


mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3,'compenscion': 4}


In [30]:
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean,retipificacion_algoritmo
0,MDM-PQR-39565361,CLIENTE SE COMUNICA POR INFORMACIÓN SOBRE SU F...,0,unicar factura concepto aclarar,4
4,MDM-PQR-39565657,NURY EDELMIRA MORENO MORENO //51563810 //60136...,0,poder generar ajuste decir parecer pagar llama...,1
6,MDM-PQR-39566135,Línea: 6013644591 Nombre: ANA BETRIZ OSORIO CU...,0,solicitar saber llegar factura tanto si decir ...,1
7,MDM-PQR-39566173,Nombre: MARTIN FLOREZ MORALES CC: 80368377 C...,0,facturacion,2
9,MDM-PQR-39566352,Nombre: JOSE DELFIN TOVAR CC: 19375096 Celular...,0,,3
...,...,...,...,...,...
3055,MDM-PQR-39736638,CLIENTE SOLICITA ACLARAR LOS CONCEPTOS FACTURA...,0,,3
3057,MDM-PQR-39736819,Línea: 6014104197 Nombre del Cliente: GUILLE...,0,parecer querer saber si aparecer,1
3059,MDM-PQR-39737146,envio de factura correcta ya que en extemprane...,0,parecer,1
3061,MDM-PQR-39737284,NombreMARIA NELLY RODRIGUEZ ANDRADE Celular 3...,0,envio facturacion ajustado,2


In [31]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename3 = f"{root_path}c_facturados_retipificacion_{current_date}.csv"



retipificacion.to_csv(filename3, index=False)





#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE
